### This Notebook will help you extract salaries from your dataframe

In [38]:
import numpy as np
import pandas as pd
import re

In [39]:
df = pd.read_csv('df_pymongo.csv')
#on prends seulement les colonnes pertinents
df = df.loc[41:]
#on reset l'index
df = df.reset_index()

In [40]:
#This is my main function that uses the other functions below. 
def add_salaries(df):
    sals = intify_etc(find_salaries(df, 'Title'))
    df['Salary'] = sals
    return df

,index,Company,Date,Details,DetailsLoc,Location,Title,_id,estimated,linked,timestamp,url,Salary
0,41,Ingeniance,il y a 3 jours,Contexte de la mission :\r\nVous assurez la pr...,Contexte de la mission :\r\nVous assurez la pr...,- Boulogne-Billancourt (92),DATA SCIENTIST - F/H\r\nIngeniance18 avis - Bo...,5c85540550a9f21dc0cf25d5,€40%C2%A0000,DATA SCIENTIST - F/H,2019-03-10,https://www.indeed.fr/emplois?q=Data%20%E2%82%...,NaN
1,42,Vekia,il y a 3 jours,Bienvenue chez Vekia !\r\nMerci mais Vekia c’e...,Bienvenue chez Vekia !\r\nMerci mais Vekia c’e...,- Paris 13e (75),Data Scientist\r\nVekia - Paris 13e (75)\r\n40...,5c85574650a9f21dc0cf25d8,€40%C2%A0000,Data Scientist,2019-03-10,https://www.indeed.fr/emplois?q=title%3ADATA%2...,42500.0
2,43,La Banque de France,il y a 3 jours,Présentation de la Direction générale\r\nVous ...,Présentation de la Direction générale\r\nVous ...,- Paris (75),DATA CONCEPTEUR - H/F\r\nLa Banque de France12...,5c85574f50a9f21dc0cf25d9,€40%C2%A0000,DATA CONCEPTEUR - H/F,2019-03-10,https://www.indeed.fr/emplois?q=title%3ADATA%2...,NaN
3,44,Cegid,il y a 3 jours,Doté(e) d'au moins 3 ans dans le développement...,Doté(e) d'au moins 3 ans dans le développement...,- Paris (75),Product Data Analyst - BI - H/F\r\nCegid36 avi...,5c85575750a9f21dc0cf25da,€40%C2%A0000,Product Data Analyst - BI - H/F,2019-03-10,https://www.indeed.fr/emplois?q=title%3ADATA%2...,NaN
4,45,Alter Solutions,il y a 3 jours,"Poste\r\n\r\nEn tant que Développeur Big Data,...","Poste\r\n\r\nEn tant que Développeur Big Data,...",- Paris (75),Développeur Big Data H/F\r\nAlter Solutions - ...,5c85575e50a9f21dc0cf25db,€40%C2%A0000,Développeur Big Data H/F,2019-03-10,https://www.indeed.fr/emplois?q=title%3ADATA%2...,52500.0
5,46,Sodicam² - Groupe Renault,il y a 3 jours,Missions et responsabilités\r\nRattaché(e) dir...,Missions et responsabilités\r\nRattaché(e) dir...,- Le Plessis-Robinson (92),Data Scientist Manager H/F\r\nSodicam² - Group...,5c85576550a9f21dc0cf25dc,€40%C2%A0000,Data Scientist Manager H/F,2019-03-10,https://www.indeed.fr/emplois?q=title%3ADATA%2...,40500.0
6,47,Total,il y a 3 jours,"Au sein de la Branche Exploration- Production,...","Au sein de la Branche Exploration- Production,...",- Courbevoie (92),"Data Analyst H/F\r\nTotal1,172 avis - Courbevo...",5c85577750a9f21dc0cf25dd,€40%C2%A0000,Data Analyst H/F,2019-03-10,https://www.indeed.fr/emplois?q=title%3ADATA%2...,NaN
7,48,SODIAAL INTERNATIONAL,il y a 3 jours,Sodiaal est né du rapprochement de 7 coopérati...,Sodiaal est né du rapprochement de 7 coopérati...,- Paris (75),DATA ANALYST H/F\r\nSODIAAL INTERNATIONAL - Pa...,5c855bd050a9f23628dc2c68,€40%C2%A0000,DATA ANALYST H/F,2019-03-10,https://www.indeed.fr/emplois?q=title%3ADATA%2...,NaN
8,49,"Pôle finances, stratégie et participations - F...",il y a 3 jours,I. ENVIRONNEMENT / CONTEXTE\r\nAu sein du dépa...,I. ENVIRONNEMENT / CONTEXTE\r\nAu sein du dépa...,- Paris (75),Modélisateur data scienti H/F\r\nPôle finances...,5c855bd750a9f23628dc2c69,€40%C2%A0000,Modélisateur data scienti H/F,2019-03-10,https://www.indeed.fr/emplois?q=title%3ADATA%2...,NaN
9,50,TEEMO,il y a 3 jours,A propos de teemo\r\nteemo est la plateforme D...,A propos de teemo\r\nteemo est la plateforme D...,- Paris 9e (75),Data Analyst (H/F) - CDI\r\nTEEMO - Paris 9e (75),5c855be150a9f23628dc2c6a,€40%C2%A0000,Data Analyst (H/F) - CDI,2019-03-10,https://www.indeed.fr/emplois?q=title%3ADATA%2...,NaN


In [32]:
def cleansal(lis):
    """cleans list of dirty salaries
    replace empty list with nan and  extracts salary
    IS USED IN FIND_SALARIES"""
    for i in range(len(lis)):
        if len(lis[i])==0:
            lis[i] = np.nan
        else:
            lis[i] = lis[i][0]
    return lis


def find_salaries(df, column):
    """trouve des salaires dans la colonne et dataframe entrees en parametres
    column doit etre un string
    renvoies une liste"""
    salaries = []
    regex = r'((?:Rémunération|Gratification|Salaire|Salary)?\s?:?\s?[0-9]*(?:.|,)[0-9]*(?:.|,)[0-9]*€?\s(?:(?:to|-|à)?\s?[0-9]*(?:.|,)[0-9]*(?:.|,)[0-9]*€?\s)?\s?(?:\/|par|per)\s?(?:mois|an|year|month))'
    for i in range(len(df)):
        l = re.findall(regex,df[column][i])
        salaries.append(l)
    salaries = cleansal(salaries) #on nettoie avec la fonction cleansal 
    return(salaries)


In [33]:
#On creeee une liste avec les salaires en string + nans quand il y en a pas (pour le colonne title seulement)
sals = find_salaries(df,'Title')

In [34]:
def intify_etc(sals):
    """paramètres : liste de salaires en forme de string
    cette fonction convertie toutes les phrases en ints: 
    remplace les salaires avec une moyenne si c'est une fourchette
    et avec une moyenne transformé en annuel s'il sagit d'une salaire par mois
    retourne une liste netoyée, prête à étre ajouté au dataframe :)) 
    """
    regex = '[0-9]*? ?[0-9]{3}'
    #for each line
    for i in range(len(sals)):
        #if it's not a nan
        if type(sals[i]) != float:
            #now we must separate per months from per year 
            #and fourchettes from non fourchettes
            if 'par an' in sals[i]:
                #print('par an')
                if '-' in sals[i]:
                    #print('fourchette')
                    frm = re.findall(regex,sals[i])[0]
                    to = re.findall(regex,sals[i])[1]
                    frm, to = frm.replace(' ',''), to.replace(' ','')#getting rid of space
                    frm, to = int(frm), int(to)#convert to int 
                    avg = (frm+to)/2 #calculate average
                    sals[i] = avg
                else:
                    #print('not fourchette')
                    sal = re.findall(regex,sals[i])[0]
                    sal=sal.replace(' ','')#get rid of space
                    sals[i] = int(sal)#convert to int
            elif 'par mois' in sals[i]:
                #print('par mois')
                if '-' in sals[i]:
                    #print('fourchette')
                    frm = re.findall(regex,sals[i])[0]
                    to = re.findall(regex,sals[i])[1]
                    frm, to = frm.replace(' ',''), to.replace(' ','')#get rid of the space
                    frm, to = int(frm), int(to)#converting to int
                    avg = (frm+to)/2#calculate average
                    par_an = avg*12#convert to yearly salary
                    sals[i] = par_an
                else:
                    #print('not fourchette')
                    sal = re.findall(regex,sals[i])[0]
                    sal = sal.replace(' ','')#get rid of space
                    sals[i] = int(sal)*12#convert to yearly salary
    return sals

sals = intify_etc(sals)

In [47]:
len(df) - df['Salary'].isna().sum()

401

In [48]:
401/2553 *100

15.707011359185271